In [4]:
from tkinter import *
import tkinter as tk
from tkinter import ttk, filedialog
import pandas as pd
import geopandas as gpd
import fiona
import threading
from tqdm import tqdm
from shapely.geometry import Point, LineString, Polygon
import sys
import io
from colorama import init, Fore

init()

# Dictionary to map color names to corresponding Tkinter tags
COLOR_TAGS = {
    "red": "tag_red",
    "blue": "tag_blue",
    "green": "tag_green",
}

# Function to update the text widget with colored text
def update_text_with_color(text_widget, message, color):
    tag_name = COLOR_TAGS.get(color)
    if tag_name:
        text_widget.config(state=tk.NORMAL)
        text_widget.insert(tk.END, message + "\n", (tag_name,))
        text_widget.config(state=tk.DISABLED)

# TextRedirector class to redirect standard output to the Text widget
class TextRedirector(io.TextIOBase):
    def __init__(self, widget, console=None):
        self.widget = widget
        self.console = console

    def write(self, text):
        self.widget.config(state=tk.NORMAL)
        self.widget.insert(tk.END, text)
        self.widget.see(tk.END)  # Scroll to the end of the text

        # Optionally, redirect the output to the original console (if provided)
        if self.console:
            self.console.write(text)

    def flush(self):
        pass  # No need to flush anything

# Function to read GeoDataFrames from the input file
def read_data(input_file, progress_update_function=None):
    # Get a list of layers available in the input file
    layers = fiona.listlayers(input_file)
    gpfs = []
    total_layers = len(layers)
    processed_layers = 0

    # Loop through each layer and read it as a GeoDataFrame
    for layer in tqdm(layers, desc="Reading GeoDataFrames"):
        gdf = gpd.read_file(input_file, layer=layer)
        gpfs.append(gdf)
        processed_layers += 1
        progress = processed_layers * 100 // total_layers
        if progress_update_function is not None:
            progress_update_function('Reading Data', layer, progress)

    return layers, gpfs

# Function to process GeoDataFrames and save results to an output file
def process_data(layers, gpfs, output_file, progress_update_function, text_widget):
    # Initialize the list of GeoDataFrames
    gdf_list = []

    

    # Process each GeoDataFrame to remove rows with action 'delete'
    for x, y in zip(layers, gpfs):
        
        num_deleted = len(y[y['action'] == 'delete'])
        if num_deleted > 0:
            message = f"Layer '{x}': {num_deleted} objects deleted since they have action equal to 'delete'"
            update_text_with_color(text_widget, message, "blue")
        
        y = y[y['action'] != 'delete']
        gdf_list.append(y)

    ############### POINT #######################
    total_layers = len(layers)
    processed_layers = 0

    # Process each GeoDataFrame and modify the geometries based on certain conditions
    for layer, gdf in zip(layers, gdf_list):
        # Counter for objects with Z value above 5000
        num_above_threshold = 0  
        
        # Check if the GeoDataFrame contains Point geometry type
        if 'Point' in gdf['geometry'].geom_type.unique():
            for geom, y in zip(gdf['geometry'], gdf['id_lokalid']):
                z_value = geom.z  # Extract the Z value from the Point geometry

                # Check if the Z value is above the threshold (5000) and update it if needed
                if z_value > 5000:
                    
                    new_z_value = -999
                    new_geom = Point(geom.x, geom.y, new_z_value)
                    gdf.loc[gdf['id_lokalid'] == y, 'geometry'] = new_geom
                    
                    # Increment the counter for each object with Z value above 5000
                    num_above_threshold += 1
                    
            
        ############################## LINESTRING #######################################
        # Check if the GeoDataFrame contains LineString or MultiLineString geometry types
        elif 'LineString' in gdf['geometry'].geom_type.unique() or 'MultiLineString' in gdf['geometry'].geom_type.unique():
            for idx, row in gdf.iterrows():
                geometry = row['geometry']

                # Check if the geometry type is 'LineString'
                if geometry.geom_type == 'LineString':
                    # Get a list of coordinates from the LineString geometry
                    coords_list = list(geometry.coords)
                    # Iterate over each coordinate in the list
                    for i, (x, y, z) in enumerate(coords_list):
                        # Check if the Z value of the coordinate is greater than 5000
                        if z > 5000:
                            # If the Z value is greater than 5000, set it to -999
                            coords_list[i] = (x, y, -999)
                    # Create a new LineString with the modified coordinates
                    modified_linestring = LineString(coords_list)

                    # Update the geometry of the GeoDataFrame at the current index with the modified LineString
                    gdf.at[idx, 'geometry'] = modified_linestring
                    
                    # Increment the counter for each object with Z value above 5000
                    num_above_threshold += 1

                elif geometry.geom_type == 'MultiLineString':
                    for linestring in geometry.geoms:
                        coords_list = list(linestring.coords)
                        for i, (x, y, z) in enumerate(coords_list):
                            if z > 5000:
                                raise Exception('MULTI-LINESTRINGS NOT ACCEPTED')
                                
           
        ################################ POLYGON ##################################
        # Check if the GeoDataFrame contains Polygon or MultiPolygon geometry types
        elif 'Polygon' in gdf['geometry'].geom_type.unique() or 'MultiPolygon' in gdf['geometry'].geom_type.unique():
            for idx, row in gdf.iterrows():
                geometry = row['geometry']

                # Check if the geometry type is 'Polygon'
                if geometry.geom_type == 'Polygon':
                    # Get a list of coordinates from the exterior of the Polygon geometry
                    coords_list = list(geometry.exterior.coords)
                    # Iterate over each coordinate in the list
                    for i, (x, y, z) in enumerate(coords_list):
                        # Check if the Z value of the coordinate is greater than 5000
                        if z > 5000:
                            # If the Z value is greater than 5000, set it to -999
                            coords_list[i] = (x, y, -999)
                    # Create a new Polygon with the modified exterior coordinates
                    modified_geometry = Polygon(coords_list)
                    # Update the geometry of the GeoDataFrame at the current index with the modified Polygon
                    gdf.at[idx, 'geometry'] = modified_geometry
                    
                    # Increment the counter for each object with Z value above 5000
                    num_above_threshold += 1

                elif geometry.geom_type == 'MultiPolygon':
                    for polygon in geometry.geoms:
                        coords_list = list(polygon.exterior.coords)
                        for i, (x, y, z) in enumerate(coords_list):
                            if z > 5000:
                                raise Exception('MULTI-POLYGONS NOT ACCEPTED')
                                
        # Print the number of objects with Z value above 5000 for the current layer in red color
        if num_above_threshold > 0:
            message = f"Layer '{layer}': {num_above_threshold} objects with abnormal Z value"
            update_text_with_color(text_widget, message, "red")

        # Save the modified GeoDataFrame to the output file in OpenFileGDB format
        gdf.to_file(output_file, layer=layer, driver="OpenFileGDB")

        processed_layers += 1
        progress = processed_layers * 100 // total_layers

        # Call the progress bar update function
        if progress_update_function is not None:
            progress_update_function('Processing data', layer, progress)

    print("Data pipeline executed successfully.")

# Function to execute the data pipeline on GeoDataFrames and save results to an output file
def run_data_pipeline(input_file, output_file):
    try:
        # Read the GeoDataFrames from the input file
        layers, gpfs = read_data(input_file)

        # Redirect the standard output to the Text widget and console
        sys.stdout = TextRedirector(output_text, sys.stdout)
        sys.stderr = TextRedirector(output_text, sys.stderr)

        # Process the GeoDataFrames
        process_data(layers, gpfs, output_file, update_progress_bar, output_text)  # Pass progress function and text widget here

    except Exception as e:
        print("Error executing data pipeline:", str(e))

# Function to check the progress of reading and data pipeline execution
def check_progress():
    global read_data_thread, pipeline_thread

    if read_data_thread.is_alive() or pipeline_thread.is_alive():
        # If either thread is still running, schedule the next update
        app.after(100, check_progress)
    else:
        # If both threads have finished, update the progress bar to 100%
        progress_var.set(100)
        progress_label.config(text="Processing complete.")
        data_pipeline_completed = True  # Update data_pipeline_completed variable
       
        # Enable the "Execute Data Pipeline" button after the pipeline is completed
        execute_button.config(state="normal", bg="blue", text="Execute Data Pipeline")
        
        # Clear input and output text fields
        output_entry.delete(0, tk.END)
        input_entry.delete(0, tk.END)

# Function to update the progress bar
def update_progress_bar(function, layer_name, progress):
    progress_var.set(progress)
    progress_label.config(text=f"{function} Layer: {layer_name} - {progress}%")
    # Update the GUI to show the progress immediately
    app.update()

# Function to browse for the input file
def browse_input():
    # Open a file dialog window to select an input file
    file_path = filedialog.askopenfilename()
    # Delete the current content of the input_entry widget
    input_entry.delete(0, tk.END)
    # Insert the selected file path into the input_entry widget
    input_entry.insert(0, file_path)

# Function to browse for the output file
def browse_output():
    # Open a file dialog window to select an output file with a default extension '.gdb'
    file_path = filedialog.asksaveasfilename(defaultextension=".gdb")
    # Delete the current content of the output_entry widget
    output_entry.delete(0, tk.END)
    # Insert the selected file path into the output_entry widget
    output_entry.insert(0, file_path)

# Function to execute the data pipeline when the button is clicked
def execute_pipeline():
    global read_data_thread, pipeline_thread

    # Get the input file path from the input_entry widget
    input_file = input_entry.get()
    # Get the output file path from the output_entry widget
    output_file = output_entry.get()

    # Redirect the standard output to the Text widget
    sys.stdout = TextRedirector(output_text)
    sys.stderr = TextRedirector(output_text)

    # Disable the "Execute Data Pipeline" button while the pipeline is running
    execute_button.config(state="disabled", bg="red", text="Running...")

    # Reset the progress bar to 0%
    progress_var.set(0)

    # Start reading data and update progress bar
    read_data_thread = threading.Thread(target=read_data, args=(input_file, update_progress_bar))
    read_data_thread.start()

    # Start the data pipeline with updating progress bar
    pipeline_thread = threading.Thread(target=run_data_pipeline, args=(input_file, output_file))
    pipeline_thread.start()

    # Check the progress of the pipeline periodically
    check_progress()

# Create the main application window
app = tk.Tk()
# Set the title of the application window to "Data Pipeline GUI"
app.title("DISPLAY FIL")

# Create a frame to hold the widgets
frm = ttk.Frame(app, padding=10)

# Create labels and entry fields for input and output file paths
input_label = tk.Label(app, text="Input File:")
input_label.pack(pady=5)  # Add 5 pixels of vertical space after the label
# Create an entry field for the input file path with a width of 50 characters
input_entry = tk.Entry(app, width=50)
input_entry.pack(pady=5)  # Add 5 pixels of vertical space after the entry field
# Create a button labeled "Browse" to open a file dialog for input file selection
input_button = tk.Button(app, text="Browse", command=browse_input)
input_button.pack(pady=5)  # Add 5 pixels of vertical space after the button

output_label = tk.Label(app, text="Output File:")
output_label.pack(pady=5)  # Add 5 pixels of vertical space after the label
# Create an entry field for the output file path with a width of 50 characters
output_entry = tk.Entry(app, width=50)
output_entry.pack(pady=5)  # Add 5 pixels of vertical space after the entry field
# Create a button labeled "Browse" to open a file dialog for output file selection
output_button = tk.Button(app, text="Browse", command=browse_output)
output_button.pack(pady=5)  # Add 5 pixels of vertical space after the button

# Create a button labeled "Execute Data Pipeline" to trigger the data pipeline execution
execute_button = tk.Button(app, text="Execute Data Pipeline", command=execute_pipeline, bg="blue", fg="white")
execute_button.pack(pady=10)  # Add 10 pixels of vertical space after the button

# Create a progress bar with a label to show the percentage
progress_var = DoubleVar()
progress_bar = ttk.Progressbar(app, variable=progress_var, maximum=100)
progress_bar.pack(pady=10)  # Add 10 pixels of vertical space after the progress bar

# Create a label to display the percentage value
progress_label = ttk.Label(app, text="0%")
progress_label.pack()

# Create a text widget to display the output
output_text = tk.Text(app, wrap="word", height=10, width=80)
output_text.pack(pady=10)  # Add 10 pixels of vertical space after the text widget

# Add tags and configure their colors
output_text.tag_configure("tag_red", foreground="red")
output_text.tag_configure("tag_blue", foreground="blue")
output_text.tag_configure("tag_green", foreground="green")


# Start the main event loop to handle user interactions and display the GUI
app.mainloop()